In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]
#訓練秒數 1S-> 0 3S-> 1 5S-> 2
trainMode = 1
#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    #loss方法
    loss_func = nn.CrossEntropyLoss()

    #資料抓入
    train, val = epg.getTrainDataset(fold, sound[trainMode], num_class)
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])

    #分批方法
    train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
    val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
    
    #訓練模型
    train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[trainMode], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
    #將gpu資料取回
    train_loss = torch.stack(train_loss,dim=-1).cpu()
    train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    val_loss = torch.stack(val_loss,dim=-1).cpu()
    val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
    for i in range(num_epochs):
        # 寫入資料
        f.write("fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    
    train_loss_array.append(train_loss)
    train_accuracy_array.append(train_accuracy)
    val_loss_array.append(val_loss)
    val_accuracy_array.append(val_accuracy)
    test1_accuracy_array.append(test1_accuracy)
    test3_accuracy_array.append(test3_accuracy)
    test5_accuracy_array.append(test5_accuracy)

f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.9406219720840454 Train_acc: 18.125000% Val_Loss: 1.8324395418167114 Val_accuracy: 31.250000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 1.6532859802246094 Train_acc: 36.250000% Val_Loss: 1.8012750148773193 Val_accuracy: 43.750000%
Test accuracy of the model: 49.200000%
Test accuracy of the model: 47.666667%
Test accuracy of the model: 45.000000%
Train Epoch: 3/30 Train_Loss: 1.5978615283966064 Train_acc: 50.000000% Val_Loss: 1.8120207786560059 Val_accuracy: 58.750000%
Test accuracy of the model: 64.000000%
Test accuracy of the model: 71.333333%
Test accuracy of the model: 72.000000%
Train Epoch: 4/30 Train_Loss: 1.4534448385

Train Epoch: 3/30 Train_Loss: 1.5755952596664429 Train_acc: 61.250000% Val_Loss: 1.4174301624298096 Val_accuracy: 61.250000%
Test accuracy of the model: 66.800000%
Test accuracy of the model: 71.666667%
Test accuracy of the model: 81.000000%
Train Epoch: 4/30 Train_Loss: 1.3584320545196533 Train_acc: 63.437500% Val_Loss: 1.578607201576233 Val_accuracy: 60.000000%
Test accuracy of the model: 62.200000%
Test accuracy of the model: 69.666667%
Test accuracy of the model: 72.000000%
Train Epoch: 5/30 Train_Loss: 1.2720309495925903 Train_acc: 63.125000% Val_Loss: 1.3080769777297974 Val_accuracy: 67.500000%
Test accuracy of the model: 63.800000%
Test accuracy of the model: 72.000000%
Test accuracy of the model: 77.000000%
Train Epoch: 6/30 Train_Loss: 1.23689866065979 Train_acc: 68.125000% Val_Loss: 1.3705805540084839 Val_accuracy: 66.250000%
Test accuracy of the model: 48.600000%
Test accuracy of the model: 54.333333%
Test accuracy of the model: 57.000000%
Train Epoch: 7/30 Train_Loss: 1.254

Train Epoch: 6/30 Train_Loss: 1.2454943656921387 Train_acc: 70.312500% Val_Loss: 1.3815081119537354 Val_accuracy: 60.000000%
Test accuracy of the model: 75.000000%
Test accuracy of the model: 83.000000%
Test accuracy of the model: 88.000000%
Train Epoch: 7/30 Train_Loss: 1.3136662244796753 Train_acc: 72.500000% Val_Loss: 1.1588040590286255 Val_accuracy: 65.000000%
Test accuracy of the model: 72.800000%
Test accuracy of the model: 82.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 8/30 Train_Loss: 1.087838888168335 Train_acc: 75.312500% Val_Loss: 1.137455940246582 Val_accuracy: 70.000000%
Test accuracy of the model: 72.600000%
Test accuracy of the model: 77.000000%
Test accuracy of the model: 85.000000%
Train Epoch: 9/30 Train_Loss: 1.0018340349197388 Train_acc: 75.937500% Val_Loss: 1.0534098148345947 Val_accuracy: 71.250000%
Test accuracy of the model: 65.600000%
Test accuracy of the model: 72.000000%
Test accuracy of the model: 79.000000%
Train Epoch: 10/30 Train_Loss: 0.9

Train Epoch: 9/30 Train_Loss: 1.1555589437484741 Train_acc: 75.000000% Val_Loss: 1.0576950311660767 Val_accuracy: 81.250000%
Test accuracy of the model: 69.000000%
Test accuracy of the model: 74.000000%
Test accuracy of the model: 84.000000%
Train Epoch: 10/30 Train_Loss: 0.9480202794075012 Train_acc: 77.187500% Val_Loss: 1.206575632095337 Val_accuracy: 80.000000%
Test accuracy of the model: 65.200000%
Test accuracy of the model: 71.333333%
Test accuracy of the model: 78.000000%
Train Epoch: 11/30 Train_Loss: 0.9187963008880615 Train_acc: 80.312500% Val_Loss: 1.1368563175201416 Val_accuracy: 82.500000%
Test accuracy of the model: 71.200000%
Test accuracy of the model: 76.000000%
Test accuracy of the model: 87.000000%
Train Epoch: 12/30 Train_Loss: 0.9897725582122803 Train_acc: 79.687500% Val_Loss: 0.9527343511581421 Val_accuracy: 82.500000%
Test accuracy of the model: 68.000000%
Test accuracy of the model: 73.333333%
Test accuracy of the model: 83.000000%
Train Epoch: 13/30 Train_Loss:

Train Epoch: 12/30 Train_Loss: 0.819581151008606 Train_acc: 79.687500% Val_Loss: 0.8497676849365234 Val_accuracy: 80.000000%
Test accuracy of the model: 83.800000%
Test accuracy of the model: 92.000000%
Test accuracy of the model: 95.000000%
Train Epoch: 13/30 Train_Loss: 0.9147161245346069 Train_acc: 79.375000% Val_Loss: 1.008399248123169 Val_accuracy: 80.000000%
Test accuracy of the model: 87.600000%
Test accuracy of the model: 94.333333%
Test accuracy of the model: 97.000000%
Train Epoch: 14/30 Train_Loss: 0.8089414834976196 Train_acc: 82.812500% Val_Loss: 1.028207540512085 Val_accuracy: 76.250000%
Test accuracy of the model: 88.200000%
Test accuracy of the model: 95.000000%
Test accuracy of the model: 97.000000%
Train Epoch: 15/30 Train_Loss: 0.8828550577163696 Train_acc: 82.187500% Val_Loss: 0.8204008936882019 Val_accuracy: 82.500000%
Test accuracy of the model: 88.400000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 97.000000%
Train Epoch: 16/30 Train_Loss: 

In [4]:
train_loss_last = []
train_accuracy_last = []
val_loss_last = []
val_accuracy_last = []
test1_accuracy_last = []
test3_accuracy_last = []
test5_accuracy_last = []

#設定存檔位置
f = open("lastData.txt", "w")

#整理出訓練完的數值
for fold in range(5):
    train_loss_last.append(train_loss_array[fold][num_epochs-1])
    train_accuracy_last.append(train_accuracy_array[fold][num_epochs-1])
    val_loss_last.append(val_loss_array[fold][num_epochs-1])
    val_accuracy_last.append(val_accuracy_array[fold][num_epochs-1])
    test1_accuracy_last.append(test1_accuracy_array[fold][num_epochs-1])
    test3_accuracy_last.append(test3_accuracy_array[fold][num_epochs-1])
    test5_accuracy_last.append(test5_accuracy_array[fold][num_epochs-1])
    # 寫入資料
    f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][num_epochs-1], train_accuracy_array[fold][num_epochs-1], val_loss_array[fold][num_epochs-1], val_accuracy_array[fold][num_epochs-1], test1_accuracy_array[fold][num_epochs-1], test3_accuracy_array[fold][num_epochs-1], test5_accuracy_array[fold][num_epochs-1]))
f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
f.close()